In [18]:
#Imported Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipyaladin import Aladin
from ipywidgets import Layout, Box, Button, HTML
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table
import os

In [19]:
target_input = input('Enter target coordinates (format: HH MM SS DD MM SS): ')

Enter target coordinates (format: HH MM SS DD MM SS): 20 58 46.99 +44 19 48.0


In [20]:
source_name=input('Enter Source Name (format: IC_348): ')

Enter Source Name (format: IC_348): NGC_7000


In [21]:
# Initialize Aladin sky viewer 
aladin = Aladin(
    layout=Layout(width="60%", height="500px"),
    target=target_input,
    fov=0.1667,  # 12 arcmin in degrees
    show_projection_control=False,
    show_fullscreen_control=True,
)


In [22]:
# Query Marton+2019 YSO catalog from Vizier around target
def fetch_ysos(center_coord_str: str, search_radius: u.Quantity = 5 * u.arcmin) -> Table:
    coord = SkyCoord(center_coord_str, unit=(u.hourangle, u.deg), frame='icrs')
    Vizier.ROW_LIMIT = -1
    catalog_id = "II/360"  # Marton+2019 YSO catalog
    columns = ["Source", "RA_ICRS", "DE_ICRS", "Jmag"]
    vizier = Vizier(columns=columns)
    result = vizier.query_region(coord, radius=search_radius, catalog=catalog_id)
    
    if len(result) == 0:
        return Table() 
    
    yso_data = result[0]
    yso_data.rename_column("Source", "GAIA_Source_ID")
    yso_data.rename_column("RA_ICRS", "RA_deg")
    yso_data.rename_column("DE_ICRS", "DEC_deg")
    return yso_data


In [23]:
# Output box 
table_output = HTML(layout=Layout(height="500px", overflow="auto"))
selected_ysos_df = pd.DataFrame()

# Button action
def handle_auto_selection():
    global selected_ysos_df

    center_coord = target_input
    yso_table = fetch_ysos(center_coord)

    if len(yso_table) == 0:
        table_output.value = "<b>No YSOs found within 6 arcmin.</b>"
        return

    records = []
    html = '<table border="1" style="border-collapse:collapse;">'
    html += "<tr><th>GAIA_Source_ID</th><th>RA (deg)</th><th>DEC (deg)</th><th>Jmag</th></tr>"
    
    for row in yso_table:
        record = {
            "GAIA_Source_ID": row["GAIA_Source_ID"],
            "RA_deg": float(row["RA_deg"]),
            "DEC_deg": float(row["DEC_deg"]),
            "Jmag": float(row["Jmag"])
        }
        records.append(record)
        html += f"<tr><td>{record['GAIA_Source_ID']}</td><td>{record['RA_deg']:.6f}</td><td>{record['DEC_deg']:.6f}</td><td>{record['Jmag']:.2f}</td></tr>"
    
    html += "</table>"
    table_output.value = html

    # Save to CSV
    selected_ysos_df = pd.DataFrame(records)
    csv_path = os.path.join(os.getcwd(), f"selected_ysos_{source_name}.csv")
    selected_ysos_df.to_csv(csv_path, index=False)
    print(f"Selected YSOs saved to: {csv_path}")


In [24]:
# Trigger auto-selection
select_button = Button(description="Auto-select YSOs (6 arcmin)")
select_button.on_click(lambda _: handle_auto_selection())


In [25]:
# Load initial catalog view in Aladin
ysos_table = fetch_ysos(target_input)
if len(ysos_table) > 0:
    aladin.add_table(ysos_table)

# Arrange all widgets in layout
layout = Box(
    children=[aladin, select_button, table_output],
    layout=Layout(display="flex", flex_flow="row", align_items="stretch", width="100%")
)

# Display the entire interface
layout


Box(children=(Aladin(layout=Layout(height='500px', width='60%')), Button(description='Auto-select YSOs (6 arcm…

Selected YSOs saved to: C:\Users\kgokh\Downloads\TAMOONS\MAIN CODES\selected_ysos_NGC_7000.csv
